### Exercise - 2 Data Augmentation and Normalization

In [34]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
import numpy as np
from torch.utils.data import DataLoader
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d


from torch.utils.tensorboard import SummaryWriter
torch.set_printoptions(linewidth=120)

In [68]:
# Hyperparameters
num_epochs = 6
num_classes = 10
batch_size = 100
learning_rate = 0.001

DATA_PATH = '/home/jaishree/Documents/DA Sem 1/DDA Lab/CNN_image_classification/cifar-10'
MODEL_PATH = '/home/jaishree/Documents/DA Sem 1/DDA Lab/CNN_image_classification/results'

In [69]:
## Data Augmentation and normalization

transform_train = transforms.Compose([
    transforms.Scale((64,64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.Scale((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


In [59]:
## Normalization

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


In [70]:
train_dataset = CIFAR10(root = DATA_PATH, download=True, transform=transform_train)
test_dataset = CIFAR10(root = DATA_PATH, train=False, transform=transform_test)

Files already downloaded and verified


In [71]:
#Data Augmentation: the process of artificially ’increasing’ our dataset by adding translation, scaling 
#and flipping to the images to fabricate examples for training.

train_len = 25*1000
part_train = torch.utils.data.random_split(train_dataset, [train_len, len(train_dataset)-train_len])[0]


train_loader = DataLoader(dataset=part_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [72]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc_layer = nn.Sequential(
            nn.Linear(15*15*128, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True)
        )
        
            
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)       
        out = out.reshape(out.size(0), -1)
        out = self.fc_layer(out)
        
        return out

In [73]:
model = ConvNet()

# Loss and opitimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [74]:
# Train the model
total_step = len(train_loader)
train_loss = []
train_acc = []

tb = SummaryWriter()

for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    for i, (images, labels) in enumerate(train_loader):
            
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss.append(loss.item())
        total_loss += loss.item()

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        total_correct += (predicted == labels).sum().item()
        train_acc.append(correct / total)

    tb.add_scalar("Both Train Loss", total_loss/len(part_train), epoch)
    tb.add_scalar("Both Train Accuracy", total_correct/len(part_train), epoch)
        
    print('Epoch [{}/{}], Step [{}/{}], training Loss: {:.4f}, training Accuracy: {:.2f}%'
              .format(epoch + 1, num_epochs, i + 1, total_step, total_loss/len(part_train),
                      (total_correct / len(part_train)) * 100))

    
tb.close()

Epoch [1/6], Step [250/250], training Loss: 0.0396, training Accuracy: 14.00%
Epoch [2/6], Step [250/250], training Loss: 0.0356, training Accuracy: 28.68%
Epoch [3/6], Step [250/250], training Loss: 0.0340, training Accuracy: 35.82%
Epoch [4/6], Step [250/250], training Loss: 0.0329, training Accuracy: 39.73%
Epoch [5/6], Step [250/250], training Loss: 0.0310, training Accuracy: 43.20%
Epoch [6/6], Step [250/250], training Loss: 0.0268, training Accuracy: 49.12%


In [75]:
# Test the model
model.eval()
test_int_acc = []
test_int_loss = []

int_correct = 0
int_total = 0
int_total_loss = 0

tb = SummaryWriter()

with torch.no_grad():
    for i, (int_images, int_labels) in enumerate(test_loader):
        int_outputs = model(int_images)
        int_loss = criterion(int_outputs, int_labels)
        test_int_loss.append(int_loss.item())
        int_total_loss += int_loss.item()

        _, predicted = torch.max(int_outputs.data, 1)
        int_total += int_labels.size(0)
        int_correct += (predicted == int_labels).sum().item()
        correct = (predicted == labels).sum().item()
        test_int_acc.append(correct / int_labels.size(0))
        
        tb.add_scalar(" Both Test Loss", int_loss.item(), i)
        tb.add_scalar("Both Test Accuracy", correct, i)

    print('Test Accuracy of the model on all test images: {} %'.format((int_correct / int_total) * 100))
    

tb.close()

Test Accuracy of the model on all test images: 49.0 %


In [16]:
p = figure(y_axis_label='Loss', width=850, y_range=(0, 5), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(train_loss)), train_loss, color='green')
p.line(np.arange(len(train_loss)), np.array(train_acc) * 100, y_range_name='Accuracy', color='red')
show(p)